import pandas!

In [ ]:
import pandas as pd

Load all the dataframes

In [ ]:
suppliers = pd.read_csv("https://www.dropbox.com/s/jjbacrux6bc1b4s/suppliers.csv?dl=1")
demand = pd.read_csv("https://www.dropbox.com/s/0yikq3pugq87vl9/demand_forecasts.csv?dl=1")
recipes = pd.read_csv("https://www.dropbox.com/s/az97voeh6i8x3i2/recipes.csv?dl=1")

In [ ]:
suppliers

In [ ]:
demand

In [ ]:
recipes

In [ ]:
#examples

In [ ]:
suppliers.loc[3]

In [ ]:
suppliers.iloc[3]

In [ ]:
suppliers = suppliers.sort_values("country_of_origin")

In [ ]:
suppliers

In [ ]:
suppliers.loc[3]

In [ ]:
suppliers.iloc[3]

In [ ]:
#Selection using masks

In [ ]:
~(suppliers["cost_per_unit"] > 1000)

In [ ]:
(suppliers["cost_per_unit"] > 1000)

In [ ]:
suppliers[ suppliers["cost_per_unit"] > 1000 ]

In [ ]:
suppliers.loc[ suppliers["cost_per_unit"] > 1000 ]

In [ ]:
(suppliers["cost_per_unit"] < 1000) & (suppliers["ingredient_uuid"] == "sugar")

In [ ]:
~((suppliers["cost_per_unit"] < 1000) | (suppliers["ingredient_uuid"] == "sugar"))

#a = it's monday
#b = it's thurday
#c = it's raining

# a & (b | c)
# (a & b) | c

In [ ]:
#operations on cols

In [ ]:
suppliers['cost_per_unit'] * suppliers['total_tax_per_unit']

In [ ]:
#oopsie
suppliers['cost_per_unit'] + suppliers['unit_name']

In [ ]:
suppliers[(suppliers['country_of_origin'] == 'denmark') | (suppliers['country_of_origin'] == 'italy')]

In [ ]:
suppliers[suppliers['country_of_origin'].isin(['denmark', 'italy'])]

In [ ]:
suppliers[suppliers['country_of_origin'].isin(['denmark', 'italy'])]

In [ ]:
suppliers['supp_id'] == "megacorp"

In [ ]:
suppliers[ suppliers['supp_id'] == "megacorp" ]

In [ ]:
#homework
demand.sort_values( ["rec_id", "demand"], ascending=True )

1) try sorting each table by different cols

what happens if you sort by a col with text in it?

2) what happens when you use loc and iloc after sorting?

try selecting the same number using loc and iloc, and see what outcome you get in each case



5) try selecting all recipies using "milk_powder"

6) try selecting all demand in the month of jan

7) try selecting all demand in the months of jan, feb, mar

4*) try setting a new index ("country_of_origin" for example) and using loc to select. This can be a little confusing at first, but it's faster in the long run if the data is big! 

clue: suppliers.set_index("ingredient_uuid").loc['hazelnut']

In [ ]:
q = suppliers.set_index("ingredient_uuid")

In [ ]:
suppliers[ suppliers['ingredient_uuid'] == "hazelnut" ]

In [ ]:
q

In [ ]:
q.loc['hazelnut']

In [ ]:
q.loc['hazelnut'].mean()

In [ ]:
##########################################################################################

In [ ]:
#nb, by default non numeric cols dropped
#nb index is now the thing we grouped by
suppliers.groupby("supp_id").mean()

In [ ]:
#min, max, median, mode, mean, var...
suppliers.groupby("supp_id").max()

In [ ]:
suppliers.groupby("ingredient_uuid").min()

In [ ]:
#nb this creates a multi index
#nb this show the min value from each column
q = suppliers.groupby(["ingredient_uuid", "supp_id"]).min()
q

In [ ]:
#this will show only one column
suppliers.groupby(["ingredient_uuid"])['time_to_deliver'].max()


so now loc works like this

In [ ]:
#to select the outer level
q.loc[ "vanilla" ]

In [ ]:
#to select both at once!
q.loc[('cocoa', 'acme_inc')]

In [ ]:
#homework

1) groupby "country_of_origin", and find the cheapest price for each ingredient

2) groupby "country_of_origin", and find the lowest tax option for each ingredient 

3) groupby "country_of_origin" and "supp_id", and find the cheapest price for each ingredient 



In [ ]:
##########################################################################################
##########################################################################################
##########################################################################################
##########################################################################################

#scenaio

Choose suppliers for each ingredient for each product.
Select the cheapest option, and the least tax option, and the fastest option.


In [ ]:
suppliers.groupby('ingredient_uuid')['cost_per_unit'].idxmin()

In [ ]:
cheapest_supplier = suppliers.loc[suppliers.groupby('ingredient_uuid')['cost_per_unit'].idxmin()]
cheapest_supplier

In [ ]:
suppliers.loc[suppliers.groupby('ingredient_uuid')['total_tax_per_unit'].idxmin()]

In [ ]:
suppliers.loc[suppliers.groupby('ingredient_uuid')['time_to_deliver'].idxmin()]

Work out total demand for each ingredient, using the demand for each product by month, and over a year.

In [ ]:
ingredient_demand = demand.merge(recipes, on='rec_id')
ingredient_demand

In [ ]:
ingredient_demand['ingredient_demand'] = ingredient_demand['demand']*ingredient_demand['qty']
ingredient_demand

In [ ]:
monthly_ingredient_demand = ingredient_demand.groupby(['ingredient_uuid','month'])[['ingredient_demand']].sum()
monthly_ingredient_demand

In [ ]:
monthly_ingredient_demand.groupby('ingredient_uuid').plot(figsize=(10,10))

In [ ]:
ingredient_demand.merge(cheapest_supplier, on='ingredient_uuid')
ingredient_demand['ingredient_demand'] = ingredient_demand['demand']*ingredient_demand['qty']
ingredient_demand

In [ ]:
demand_by_supply = ingredient_demand.merge(cheapest_supplier, on='ingredient_uuid')
demand_by_supply['ingredient_cost'] = demand_by_supply['ingredient_demand']*demand_by_supply['cost_per_unit']
demand_by_supply

Work out total cost per unit to product


In [ ]:
demand_by_supply.groupby('rec_id')['ingredient_cost'].sum()

In [ ]:
demand_by_supply.groupby('rec_id')['ingredient_cost'].sum().plot.bar()

(Homework) work out how to find the total cost of goods needed for each producer country.


In [ ]:
ingredient_demand.merge(cheapest_supplier, on="ingredient_uuid").groupby('country_of_origin').sum()

(Homework) work out how to find the total tax, using the cheapest supplier for each ingredient, to create 1000 of each recipe.

In [ ]:
q = ingredient_demand.merge(cheapest_supplier, on="ingredient_uuid").groupby('ingredient_uuid').sum()
q[['total_tax_per_unit']]

In [ ]:
q['total_tax_per_unit'].plot.bar(x = "ingredient_uuid", y="total_tax_per_unit")

In [ ]:
q['total_cost'] = q['ingredient_demand']*q['cost_per_unit'] + q['ingredient_demand']*q['total_tax_per_unit']
q

In [ ]:
q['total_cost'].plot.bar()

In [ ]:
q[['total_cost', 'time_to_deliver']].plot.scatter(x='total_cost', y='time_to_deliver')

In [ ]:
q = ingredient_demand.merge(cheapest_supplier, on="ingredient_uuid").groupby('ingredient_uuid')[['total_tax_per_unit']].sum()
q

In [ ]:
#Day 5

In [ ]:
from random import randint
import numpy as np

In [ ]:
df = pd.read_csv("https://www.dropbox.com/s/vz1l18qa9xxvhy3/demand_trend.csv?dl=1", index_col=0)

df['date'] = pd.to_datetime(df['date'])
q = df[ ['date', 'demand'] ]
q.plot(x='date', y='demand', figsize=(10,10))

In [ ]:
q

In [ ]:
q.dropna()

In [ ]:
df.fillna( df['demand'].mean() )

In [ ]:
df.fillna( df['demand'].mean() )['demand'].plot()

In [ ]:
df.fillna( method='ffill' )

In [ ]:
df.fillna( method='bfill' )

In [ ]:
df.fillna( method='ffill' ).fillna( method='bfill' )

In [ ]:
df.fillna( method='ffill' ).fillna( method='bfill' )['demand'].plot()

In [ ]:
df['day_of_year'] = df['date'].apply( lambda x : x.dayofyear )

df['week_of_year'] = df['date'].apply( lambda x : x.weekofyear )

df['day_of_week'] = df['date'].apply( lambda x : x.day_of_week )

df['is_month_start'] = df['date'].apply( lambda x : x.is_month_start )

df['month'] = df['date'].apply( lambda x : x.month )

In [ ]:
df

In [ ]:
df.fillna(df['demand'].mean()).groupby( 'week_of_year' )['demand'].sum().plot()

In [ ]:
df.fillna(df['demand'].mean()).groupby( 'day_of_week' )['demand'].sum().plot.bar()

In [ ]:
#homework : create a bar chart grouping all the data by month. Is any one month significantly higher than the others

In [ ]:
#homework (research needed) : use diff + shift functions to work out rate of demand growth month/month and year / year
#take care to notice what happens with nan

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [ ]:
cal = calendar()
holidays = cal.holidays(start=df['date'].min(), end=df['date'].max())

df['holiday'] = df['date'].isin(holidays)

In [ ]:
df.groupby('holiday')['demand'].mean().plot.bar()

In [ ]:
df['diff_day'] = df.fillna( method='ffill' ).fillna( method='bfill' )['demand'].diff(periods=1)


In [ ]:
df.fillna( method='ffill' ).fillna( method='bfill' )['demand'].diff(periods=10)

In [ ]:
df

In [ ]:
df['demand_shift'] = df['demand'].shift(2)
df

In [ ]:
df['dod_demand'] = df['demand'] / df['demand_shift']
df